In [1]:
import math
import string
import time
from collections import defaultdict
from helpful_functions import *
import re
import copy
import functools

In [2]:
with open("day19_input.txt", "r") as f:
    prog_inp = f.readlines()

prog_inp = [line.rstrip() for line in prog_inp]

for i in range(len(prog_inp)-1,-1,-1):
    if prog_inp[i] == '': del prog_inp[i]
    else: break
print(prog_inp[:5])

['17: 72 112 | 71 3', '31: 71 33 | 72 57', '49: 72 110 | 71 45', '37: 71 68 | 72 45', '124: 71 128 | 72 64']


In [3]:
(rules_unparsed, messages) = group_by(prog_inp, '')
# print(rules_unparsed)
# print(messages)

rules = dict()
for r in rules_unparsed:
    tmp = r.split(": ")
    rule_name = int(tmp[0])
    rule_body = tmp[1]
    if 'a' in rule_body or 'b' in rule_body:
        rule_body = rule_body[1]
    else:
        rule_body = tuple(tuple(int(i) for i in o.split(' ')) for o in rule_body.split(" | "))
    rules[rule_name] = rule_body

# reverse_rules_per_rule = defaultdict(lambda: set())
# reverse_rules_by_combination = defaultdict(lambda: set())

# for k,v in rules.items():
#     if type(v) == str:
#         reverse_rules_per_rule[v] = k
#         reverse_rules_by_combination[v] = k
#     else:
#         for legal_combination in v:
#             reverse_rules_by_combination[tuple(legal_combination)].add(k)
#             for sub_rule in legal_combination:
#                 reverse_rules_per_rule[sub_rule].add(k)

# print(rules)
# print(reverse_rules_per_rule)
# print(reverse_rules_by_combination)

In [4]:
new_rules = copy.deepcopy(rules)
new_rules[8] = ((42,), (42, 8,),)
new_rules[11] = ((42, 31,), (42, 11, 31,),)

In [5]:
# This worked for the test cases but crashed my computer on the
# real input because it was too slow/ate too much memory

def combine(current_message, i=0, ismain=False):
    current_message = copy.deepcopy(current_message)
    while i < len(current_message):
        if i + 1 < len(current_message):
            for new in reverse_rules_by_combination[(current_message[i], current_message[i+1])]:
                new_msg = current_message[:i] + [new] + current_message[i+2:]
                if combine(new_msg, i-1):
                    return True
        i += 1
    print(current_message)
    if current_message == [0]:
        return True
    return False

def is_valid(message):
    message = [reverse_rules_per_rule[l] for l in list(message)]
    return combine(message, ismain=True)

# total_valid = 0
# for i, msg in enumerate(messages):
#     # if i%20 == 0:
#     print(i)
#     if is_valid(msg):
#         total_valid += 1

# print("total valid:", total_valid)

In [6]:
def break_apart(current_rule, message, offsets, depth=1, use_new_rules=True):
    # print("|"*depth, current_rule, message, offsets)
    if use_new_rules:
        subrules = new_rules[current_rule]
    else:
        subrules = rules[current_rule]
    
    # print("|"*depth, subrules)

    if type(subrules) == str:
        new_offsets = []
        for offset in offsets:
            if offset < len(message):
                if subrules == message[offset]:
                    new_offsets.append(offset+1)
        if len(new_offsets) == 0:
            return False
        return new_offsets

    new_offsets = []
    for i, legal_combination in enumerate(subrules):
        potential_offsets = offsets
        valid = True
        for j, rule in enumerate(legal_combination):
            if (potential_offsets := break_apart(rule, message, potential_offsets, depth+1)) == False:
                valid = False
                break
        if valid:
            new_offsets += potential_offsets

    if len(new_offsets) == 0:
        return False
    return new_offsets


def is_valid(message):
    res = break_apart(0, message, [0])
    if res == False:
        return False
    elif len(message) in res:
        return True
    else:
        return False

total = 0
for msg in messages:
    if is_valid(msg):
        total += 1

print("total:", total)

total: 228
